In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, RepeatVector, Dot, Activation, LSTM, Concatenate, Bidirectional, Input
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.optimizers import Adam
import random
from nmt_utils import *

In [4]:
m = 10000
dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(m)

100%|██████████| 10000/10000 [00:00<00:00, 46032.67it/s]


In [5]:
dataset[:10]

[('9 may 1998', '1998-05-09'),
 ('10.11.19', '2019-11-10'),
 ('9/10/70', '1970-09-10'),
 ('saturday april 28 1990', '1990-04-28'),
 ('thursday january 26 1995', '1995-01-26'),
 ('monday march 7 1983', '1983-03-07'),
 ('sunday may 22 1988', '1988-05-22'),
 ('08 jul 2008', '2008-07-08'),
 ('8 sep 1999', '1999-09-08'),
 ('thursday january 1 1981', '1981-01-01')]

In [6]:
human_vocab

{' ': 0,
 '.': 1,
 '/': 2,
 '0': 3,
 '1': 4,
 '2': 5,
 '3': 6,
 '4': 7,
 '5': 8,
 '6': 9,
 '7': 10,
 '8': 11,
 '9': 12,
 'a': 13,
 'b': 14,
 'c': 15,
 'd': 16,
 'e': 17,
 'f': 18,
 'g': 19,
 'h': 20,
 'i': 21,
 'j': 22,
 'l': 23,
 'm': 24,
 'n': 25,
 'o': 26,
 'p': 27,
 'r': 28,
 's': 29,
 't': 30,
 'u': 31,
 'v': 32,
 'w': 33,
 'y': 34,
 '<unk>': 35,
 '<pad>': 36}

In [7]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [8]:
Tx = 30
Ty = 10
X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)

print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

X.shape: (10000, 30)
Y.shape: (10000, 10)
Xoh.shape: (10000, 30, 37)
Yoh.shape: (10000, 10, 11)


In [9]:
index = 0
print("Source date:", dataset[index][0])
print("Target date:", dataset[index][1])
print()
print("Source after preprocessing (indices):", X[index])
print("Target after preprocessing (indices):", Y[index])
print()
print("Source after preprocessing (one-hot):", Xoh[index])
print("Target after preprocessing (one-hot):", Yoh[index])

Source date: 9 may 1998
Target date: 1998-05-09

Source after preprocessing (indices): [12  0 24 13 34  0  4 12 12 11 36 36 36 36 36 36 36 36 36 36 36 36 36 36
 36 36 36 36 36 36]
Target after preprocessing (indices): [ 2 10 10  9  0  1  6  0  1 10]

Source after preprocessing (one-hot): [[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
Target after preprocessing (one-hot): [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


<table>
<td> 
<img src="images/attn_model.png" style="width:500;height:500px;"> <br>
</td> 
<td> 
<img src="images/attn_mechanism.png" style="width:500;height:500px;"> <br>
</td> 
</table>

In [10]:
repeater = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(activation='tanh', units=10)
densor2 = Dense(activation='relu', units=1)
activator = Activation(softmax, name='attentions_weights')
dotor = Dot(axes=1)

In [11]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attention) LSTM cell
    """
    s = repeater(s_prev)
    x = concatenator([a,s])
    x = densor1(x)
    x = densor2(x)
    x = activator(x)
    return dotor([x, a])

In [12]:
n_a = 32
n_s = 64

post_activation_LSTM_cell = LSTM(units=n_s, return_state=True)
output_layer = Dense(units=len(machine_vocab), activation=softmax)

In [13]:
def modelf(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "human_vocab"
    machine_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
    model -- Keras model instance
    """
    X = Input(shape=(Tx,human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    outputs = []

    a = Bidirectional(LSTM(units=n_a, return_sequences=True))(X)
    for t in range(Ty):
        context = one_step_attention(a, s)
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, c])
        output = output_layer(s)
        outputs.append(output)
    return Model(inputs=[X, s0, c0], outputs=outputs)


In [14]:
model = modelf(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 30, 37)]     0           []                               
                                                                                                  
 s0 (InputLayer)                [(None, 64)]         0           []                               
                                                                                                  
 bidirectional (Bidirectional)  (None, 30, 64)       17920       ['input_1[0][0]']                
                                                                                                  
 repeat_vector (RepeatVector)   (None, 30, 64)       0           ['s0[0][0]',                     
                                                                  'lstm[0][0]',               

In [16]:
opt = Adam(learning_rate=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [17]:
s0 = np.zeros((m, n_s))
c0= np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0, 1))

In [18]:
model.fit([Xoh, s0, c0], outputs, epochs=1, batch_size=100)

100/100 [==============================] - 16s 33ms/step - loss: 17.0342 - dense_2_loss: 1.2090 - dense_2_1_loss: 0.9793 - dense_2_2_loss: 1.7965 - dense_2_3_loss: 2.7027 - dense_2_4_loss: 0.8899 - dense_2_5_loss: 1.3623 - dense_2_6_loss: 2.6893 - dense_2_7_loss: 1.0665 - dense_2_8_loss: 1.7323 - dense_2_9_loss: 2.6062 - dense_2_accuracy: 0.5130 - dense_2_1_accuracy: 0.7098 - dense_2_2_accuracy: 0.2888 - dense_2_3_accuracy: 0.0955 - dense_2_4_accuracy: 0.8057 - dense_2_5_accuracy: 0.3229 - dense_2_6_accuracy: 0.0514 - dense_2_7_accuracy: 0.8485 - dense_2_8_accuracy: 0.2516 - dense_2_9_accuracy: 0.1077


In [19]:
# Loading Pretrained Weights. [Trained for over 1000 epochs on RTX-4060Ti 8G]
model.load_weights("./SPN.keras")

In [22]:
human_date = "Sep 29, 1979"
x = string_to_int(human_date, Tx, human_vocab)
x = tf.one_hot(x, len(human_vocab))
x = tf.expand_dims(x, 0)

In [23]:
s0 = np.zeros((1, n_s))
c0 = np.zeros((1, n_s))

pred = np.argmax(model.predict([x, s0, c0]), axis=-1)
pred = [int(i) for i in pred]
pred = int_to_string(pred, inv_machine_vocab)
print("".join(pred))


1/1 [==============================] - 0s 33ms/step
1979-09-29
